In [1]:
# importing required libraries
import dash
from dash import dcc
import dash_bootstrap_components as dbc
from dash import html
from dash.dependencies import Input, Output

import plotly.graph_objects as go

import json
import os
from glob import glob
from pathlib import Path
import pandas as pd

import src.plotlyplot as plot
import src.elevation as elevation

In [2]:
fig_height_point = 150
fig_height_sec = 150

In [3]:
liste_multi = ["Panneaux", "Usages", "Revetement"]
liste_bisses = ["Clavau", "Sion", "Grand bisse d'Ayent"]

In [4]:
bisses_dict = {}
selected_bisse_json = {}
data_plot = {}
elevation_profile = (
    None  # df of elevation profile, so no need to load it again at each update
)
range_el = []  # will be len 2


def get_list_bisse():
    global bisses_dict

    data = "data"
    PATH = os.path.join(data, "bisses")

    # get all json of bisse
    # folders = [f.path for f in os.scandir(PATH) if f.is_dir()]
    # files = [y for x in os.walk(PATH) for y in glob(os.path.join(x[0], "*.json"))]
    # filenames = [Path(path).stem for path in files]

    files = []
    filenames = []

    for item in os.scandir(PATH):
        if item.is_dir():
            for file in os.scandir(item):
                if file.name.endswith(".json"):
                    files.append(os.path.split(file.path)[0])
                    filenames.append(file.name)

    print(files)
    print(filenames)

    # ! check if file is missing for a folder
    # all bisse where data has been collected in a json
    dic = dict(zip(filenames, files))
    bisses_dict = dic
    return dic.keys()


def get_list_ind(bisse):
    global data_plot

    PATH = bisses_dict[bisse]

    couche_liste = [
        "environnement",
        "touristique",
        "social",
        "patrimoine",
        "hydrologique",
    ]

    all_json = {}
    for couche in couche_liste:
        try:
            with open(os.path.join(PATH, couche, "plot.json"), "r") as file:
                plot_json = json.load(file)
                all_json.update(plot_json)
                # print(plot_env)
        except FileNotFoundError:
            print("No plotfile for couche " + couche)

    data_plot = all_json
    return [
        {"label": entry["nom_indice"], "value": key} for key, entry in all_json.items()
    ]

In [5]:
# app = dash.Dash()
app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

In [6]:
def get_cc_graph(x_trucks=[5, 4, 7, 4, 8], y_ships=[6, 3, 5, 3, 7], id="id"):
    return dcc.Graph(
        id=id,
        figure={
            "data": [
                {
                    "x": [1, 2, 3, 4, 5],
                    "y": x_trucks,
                    "type": "line",
                    "name": "Trucks",
                },
                {
                    "x": [1, 2, 3, 4, 5],
                    "y": y_ships,
                    "type": "bar",
                    "name": "Ships",
                },
            ],
            "layout": {"title": "Basic Dashboard"},
        },
    )

In [7]:
button = html.Button(children="Button")

col_selection = html.Div(
    [
        html.Div(
            [
                html.H3(children="Selections"),
                html.Div(children="Bisse: "),
                dcc.Dropdown(
                    id="bisse_dd",
                    options=[{"label": m, "value": m} for m in get_list_bisse()],
                    multi=False,  # Or set to False
                    clearable=True,
                    disabled=False,
                ),
                html.Div(children="Indicateurs: "),
                dcc.Dropdown(
                    id="indicateur_dd",
                    multi=True,
                    options=[{"label": m, "value": m} for m in liste_multi],
                    clearable=True,
                    disabled=False,
                ),
            ]
        ),
    ]
)

col_profile = html.Div(
    children=[get_cc_graph(id="1"), get_cc_graph(id="2"), get_cc_graph(id="3")],
    id="graph_list",
)


col_val_uniques = html.Div(
    children=[
        dcc.Input(id="input", value="Enter a number", type="text"),
        html.Div(id="output"),
    ]
)

multi_fonction = html.Div(
    [
        dbc.Row(
            [
                html.H1(
                    "Nom du Bisse",
                    style={"textAlign": "center", "color": "red", "fontSize": 40},
                )
            ]
        ),
        dbc.Row(
            [
                dbc.Col(col_selection, width=2),
                dbc.Col(col_profile, width=8),
                dbc.Col(col_val_uniques, width=2),
            ]
        ),
    ]
)

['data\\bisses\\clavau']
['clavau.json']


In [8]:
@app.callback(
    Output(component_id="output", component_property="children"),
    [Input(component_id="input", component_property="value")],
)
def update_value(input_data):
    try:
        return str(float(input_data) ** 2)
    except:
        return "Error, the input is not a number"


@app.callback(
    Output(component_id="indicateur_dd", component_property="options"),
    [Input(component_id="bisse_dd", component_property="value")],
)
def update_list_ind(input_data):
    global elevation_profile, selected_bisse_json, range_el

    # handle at start up
    if input_data == None:
        return []
    # load bisses json
    # print("Filename of bisse json")
    # print(bisses_dict[input_data] + input_data)
    with open(bisses_dict[input_data] + "\\" + input_data, "r") as file:
        selected_bisse_json = json.load(file)
    # calculate elevation profile when new bisse is selected

    elevation_profile = elevation.get_elevation_from_inventaire(
        selected_bisse_json["nom_inventaire"]
    )

    range_el = [elevation_profile["dist_m"].min(), elevation_profile["dist_m"].max()]
    # return list of dcc graphs
    return get_list_ind(input_data)


@app.callback(
    Output(component_id="graph_list", component_property="children"),
    [Input(component_id="indicateur_dd", component_property="value")],
)
def update_graphs(input_data):
    # is a list
    graph_list = []
    # handle at start up
    if input_data == None:
        return [get_cc_graph(id="1"), get_cc_graph(id="2"), get_cc_graph(id="3")]
    print(input_data)

    figure_elevation = go.Figure()
    plot.plot_elevation(figure_elevation, elevation_profile)

    elevation_graph = dcc.Graph(
        id="elevation_graph", figure=figure_elevation, config=plot.general_config
    )
    graph_list.append(elevation_graph)

    # plot.plot_elevation(axs[0], bisse_profile)
    # axs[0].set_title("Profile d'élevation")

    for i, indicateur in enumerate(input_data):
        # json of data to plot
        indicateur_json = data_plot[indicateur]
        data = pd.read_csv(indicateur_json["path"])

        new_fig = go.Figure()

        display(indicateur_json)

        if indicateur_json["type_donnee"] == "section":
            plot.plot_sections(new_fig, data, range_el, fig_height=fig_height_sec)
        elif indicateur_json["type_donnee"] == "point":
            plot.plot_point_data(
                new_fig,
                data,
                range_el=range_el,
                fig_height=fig_height_point,
                num_bins=50,
            )
        elif indicateur_json["type_donnee"] == "autre":
            print("can't find implementation of plot for that type of data")
        else:
            print("Unknown data type")

        # ! do all styling here instead of in each plot function
        #! or make a function on plotlyplot.py that applies the same configs to all figure

        new_graph = dcc.Graph(
            id="grap_" + str(i),
            figure=new_fig,
            config=plot.general_config,
        )
        graph_list.append(new_graph)

    # return list of dcc graphs
    return graph_list

## Placeholders

In [9]:
gestion_eau = html.Div(
    children=[
        html.H1("Dash Tutorial"),
        dcc.Graph(
            id="example",
            figure={
                "data": [
                    {
                        "x": [1, 2, 3, 4, 5],
                        "y": [5, 4, 7, 4, 8],
                        "type": "line",
                        "name": "Trucks",
                    },
                    {
                        "x": [1, 2, 3, 4, 5],
                        "y": [6, 3, 5, 3, 7],
                        "type": "bar",
                        "name": "Ships",
                    },
                ],
                "layout": {"title": "Basic Dashboard"},
            },
        ),
    ]
)

inventaire = [
    html.Div(
        [
            html.Div("Convert Temperature"),
            "Celsius",
            dcc.Input(id="celsius-2", value=0.0, type="number"),
            " = Fahrenheit",
            dcc.Input(
                id="fahrenheit-2",
                value=32.0,
                type="number",
            ),
        ]
    ),
]

In [10]:
tab_pages = [
    dcc.Tab(label="Multi-fonctio", children=multi_fonction),
    dcc.Tab(label="Gestion de l'eau", children=gestion_eau),
    dcc.Tab(label="Inventaire", children=inventaire),
]
children = html.Div([dcc.Tabs(id="tabs", children=tab_pages)])

In [11]:
app.layout = children

In [ ]:
app.run(jupyter_mode="external", port=8000)

Dash app running on http://127.0.0.1:8000/


No plotfile for couche social
No plotfile for couche patrimoine
No plotfile for couche hydrologique
['biotope']


{'nom_indice': 'Biotopes',
 'type_donnee': 'section',
 'format_donnee': ['categorique'],
 'drop_down': 'path/to/file',
 'col': [['val_av', 'val_ap']],
 'path': 'data\\bisses\\clavau\\environnement\\Biotopes\\biotope.csv'}

[]
['didact']


{'nom_indice': 'Panneaux information',
 'type_donnee': 'point',
 'format_donnee': ['categorique'],
 'drop_down': 'path/to/file',
 'col': ['type'],
 'path': 'data\\bisses\\clavau\\touristique\\Panneaux information\\didact.csv'}

['didact', 'biotope']


{'nom_indice': 'Panneaux information',
 'type_donnee': 'point',
 'format_donnee': ['categorique'],
 'drop_down': 'path/to/file',
 'col': ['type'],
 'path': 'data\\bisses\\clavau\\touristique\\Panneaux information\\didact.csv'}

{'nom_indice': 'Biotopes',
 'type_donnee': 'section',
 'format_donnee': ['categorique'],
 'drop_down': 'path/to/file',
 'col': [['val_av', 'val_ap']],
 'path': 'data\\bisses\\clavau\\environnement\\Biotopes\\biotope.csv'}